## Description

This notebook will extract training data from the ODC using geometries within a geojson. The dataset will use the NNI level labels within the 'data/nni_training_spain.geojson' file.

In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs

import sys
sys.path.insert(1, '../')
from tools.plotting import map_shapefile
from tools.bandindices import calculate_indices
from tools.datahandling import mostcommon_crs
from tools.classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'dask_ml'

In [ ]:
#Connect to the datacube
dc = datacube.Datacube(app='Sentinel-2')

## Analisis parameters
* path : The path to the input vector file from witch we wil extract training data.
* field : This is the name of the columns in your shapefile attribute table that contains the class lables. The class lables must be integers.

In [ ]:
path = 'data/nni_training_spain.geojson' 
field = 'class'

# Find the number of CPU's

In [ ]:
##ncpus = round(get_cpu_quota()) we can use this to find the number of cpu we have, we are going to set it to 2
ncpus = 2;
print('ncpus = ' + str(ncpus))

# Preview input data
We can load and preview our input data shapefile using geopandas.

In [ ]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

In [ ]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

# Extracting training data
The function collect_training_data takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any NaN or inf values.The below variables can be set within the collect_training_data function:

* zonal_stats: An optional string giving the names of zonal statistics to calculate across each polygon (if the geometries in the vector file are polygons and not points). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.

In addition to the zonal_stats parameter, we also need to set up a datacube query dictionary for the Open Data Cube query such as measurements (the bands to load from the satellite), the resolution (the cell size), and the output_crs (the output projection). These options will be added to a query dictionary that will be passed into collect_training_data using the parameter collect_training_data(dc_query=query, ...). The query dictionary will be the only argument in the feature layer function which we will define and describe in a moment.

In [ ]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
# Create a reusable query
query = {
    'time': ('2022'),
    'resolution': (-30, 30),
    'measurements': ['red', 'green', 'blue', 'red_edge_1', 'red_edge_2', 'red_edge_3', 'nir']
}

# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product='s2_l2a', query=query)

## Defining feature layers
To create the desired feature layers, we pass instructions to collect_training_data through the feature_func parameter.

feature_func: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The feature_func must accept a dc_query dictionary, and return a single xarray.Dataset or xarray.DataArray containing 2D coordinates (i.e x, y - no time dimension). e.g.

    def feature_function(query):
        dc = datacube.Datacube(app='feature_layers')
        ds = dc.load(**query)
        ds = ds.mean('time')
        return ds

In [ ]:
def feature_layers(dc, query):
    #load s2 annual geomedian
    ds = dc.load(product='gm_s2_annual',
                 **query)
    
    #calculate some band indices
    ds = calculate_indices(ds,
                           index=['NDVI'],
                           drop=True,
                           satellite_mission='s2')
    
    return ds

Run the collect_training_data function

In [ ]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

In [ ]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))